<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3>
<h4 align="center"><a href="mailto:echen29@uwo.ca"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Robarts Research Institute, London</h4>
<h4 align="center">Department of Electrical and Computer Engineering, Western University</h4>
<h4 align="center">School of Biomedical Engineering, Western University</h4>
<h4 align="center">Department of Medical Biophysics, Western University</h4>
<h4 align="center">Day 06, January 22, 2019</h4>

### Introduction

By we are have build a few utility functions to assists us in terms of segmentatio and visualization. We also have performed a simple thresholding example based on a chess phantom and imaged in CT. The reason a simple, binary, thresholding technique works well for this phantom (and the fact it is in CT) is because the phantom was made of a single (and dense) material. In CT, it has the characteristic of being bright (i.e. high intensity value) as compared to other objects in the image volume.  The fact that the phantom-bone material was [homogeneous](https://en.oxforddictionaries.com/definition/homogeneous) and solid helped the segmentation process as well.

Real tissue, however, is not homogeneous in structure. Take a long [bone](https://en.wikipedia.org/wiki/Bone) such as the femur, as an example, it contains the 
* [cortical bone](https://en.wikipedia.org/wiki/Bone#Cortical_bone): the hard outer layer of bones
* [Cancellous bone](https://en.wikipedia.org/wiki/Bone#Cancellous_bone): the internal tissue of the skeletal bone and is an open cell porous network

<a title="By Pbroks13 (Own work) [CC BY 3.0 (http://creativecommons.org/licenses/by/3.0)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3ABone_cross-section.svg"><img width="512" alt="Bone cross-section" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Bone_cross-section.svg/512px-Bone_cross-section.svg.png"/></a>

As such, the appearance of bone in CT is not homogeneous either:
![test](https://openi.nlm.nih.gov/imgs/512/331/3460560/PMC3460560_ci12904902.png)
(Credit of the image goes to [Vanel et al.](https://openi.nlm.nih.gov/detailedresult.php?img=PMC3460560_ci12904902&query=&req=4&simCollection=PMC3460560_ci12904902&npos=1))

A simple (binary) thresholding technique may not work in these cases. Fortunately, SimpleITK has many types of segmentation filters that will facilitate us in these scenario.

In this Jupyter Notebook, we'll look into some of the image segmenation filters available in SimpleITK that will partition an image into (hopefully) meaningful regions. The goal of this notebook is to become familiar with basic segmentation algorithms avilable in SimpleITK, and interactively explore their parameter space.

The **output** of the segmentation pipeline is commonly an **image** of integers where each integer can represent an object.  This is commonly as a **label map**.  In this **image**, the value `0` is often used to denote the background, and 1 (sometimes 255) for a foreground object.

In [ ]:
from __future__ import print_function
#
# By now we should know how to import the necessary modules into the Jupyter Notebooks/Python. It is a good practice
# to import all modules at the beginning of your python script.
#

# for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# widgets for interactions
from ipywidgets import interact, FloatSlider, fixed


# SimpleITK
import SimpleITK as sitk

# Download data to work on
%run update_path_to_download_script
from downloaddata import fetch_data as fdata

# utilities
import os

# utilities for diplaying images
from myshow import myshow, myshow3d

Let us get some data first. For the purpose of segmentation, load the CT of the chess phantom

In [ ]:
img_ct = sitk.ReadImage(fdata("cthead1.png"))

In [ ]:
help(sitk.StatisticsImageFilter)

In [ ]:
stats = sitk.StatisticsImageFilter()
stats.Execute(img_ct)
print('minimum:', stats.GetMinimum())
print('maximum:', stats.GetMaximum())
print('mean:', stats.GetMean())
print('std.:', stats.GetSigma())

In [ ]:
myshow(img_ct)
imgPixelID = img_ct.GetPixelID()
# print(img_ct.GetPixelID())
print(img_ct.GetPixelIDTypeAsString())

Note that the file we just read is of `.png` format: as such, the range of the pixel value is already scaled for us to be within [0...255] value. The use of `8-bit unsigned integer` is a hint too. 

Remember, the [CT Hounsfield Unit](https://en.wikipedia.org/wiki/Hounsfield_scale) for for 
* distilled at water at standard pressure and temperature (STP) is `0` **by definition**,
* air is around `-1000`,
* bone:
  * +200 in craniofacial bone
  * +700 in cancellous bone
  * +3000 in cortical bone

Mapping the intensity value from Hounsfield Unit (UH) to certain range can be achieved using a simple linear transformation:

$outputPixel = (inputPixel-inputMin) \cdot \frac{(outputMax - outputMin)}{(inputMax - inputMin)} + outputMin$

Suppose in the original CT image a pixel corresponding to the cortical bone has an intensity value of $3000$. The min/max pixel intensity value of the CT image is $-1500$ and $3500$, respectively. If we are to save it as a [`.PNG`](https://en.wikipedia.org/wiki/Portable_Network_Graphics) file format, where each pixel is stored as a 25-bit RGB or 32-bit RGBA color (8-bit per channel), the linearly mapped output pixel intensity is:

8-bit implies ($2^8=256$) grayscale values

$x = (3000 - (-1500)) \cdot \frac{255-0}{3500 - (-1500)} + 0 = 230$

This mapping can be done using [sitk.RescaleIntensityImageFilter](https://itk.org/Doxygen/html/classitk_1_1RescaleIntensityImageFilter.html).

## Thresholding

For the sake of completeness, let us look at thresholding again as it is the most basic form of segmentation. It simply labels the pixels of an image based on the intensity range without respect to geometry or connectivity.

In [ ]:
# plt.hist(sitk.GetArrayViewFromImage(vol)[z,:,:])
plt.figure()
plt.hist(sitk.GetArrayViewFromImage(img_ct))
plt.title("Histogram")
plt.show()

In [ ]:
# to visualize the labels image in RGB needs an image with 0-255 range
#
# we don't need to do this for THIS particular image (it is already rescaled).
img_255 = sitk.Cast(sitk.RescaleIntensity(img_ct), sitk.sitkUInt8)

The reason we need to create another image `img_255` is for visualization purposes. A label map (image) is simply an image of integers where each integer value denotes a label. It can be displayed as an RGB picture. As such, using the `sitk.LabelOverlay` function the CT image itself needs to be scaled as well.

In [ ]:
seg = img_ct > 220
myshow(sitk.LabelOverlay(img_255, seg), "Basic Thresholding")

In [ ]:
# what doees LabelOverlay do?
help(sitk.LabelOverlay)

In [ ]:
# We performed a simple arithmetic operation on the SimpleITK Image class. What is the output of that operation?
#help(seg)
print(seg.GetPixelIDTypeAsString())

While arithemtic operation allows us to perform a thresholding operation, the SimpleITK `BinaryThreshold` class perform thresholding operation with an [lower, upper] bound, and we can assign a value (label) for both inside and outside range of the threshold values.

In [ ]:
seg = sitk.BinaryThreshold(img_ct, lowerThreshold=130, upperThreshold=155, insideValue=1, outsideValue=0)
myshow(sitk.LabelOverlay(img_255, seg), "Binary Thresholding")

We instintively choose threshold values based on the histogram of the image. SimpleITK has a number of histogram based thresholding filters that automatically selects a threholding values based on the histogram statistics, such the [maximum entropy](https://www.sciencedirect.com/science/article/pii/0734189X85901252) and the popular [Otsu's method](http://ieeexplore.ieee.org/document/4310076/).

[Otsu's method](http://ieeexplore.ieee.org/document/4310076/), for example, assumes that the image contains two classes of pixel following a [bi-model](https://en.wikipedia.org/wiki/Multimodal_distribution) distribution (background and foreground). It then automatically calculates the optimum threshold separating the two classes so that their combined spread (intra-class variance) is minimal.

In [ ]:
otsu_filter = sitk.OtsuThresholdImageFilter()
otsu_filter.SetInsideValue(0)
otsu_filter.SetOutsideValue(1)
seg = otsu_filter.Execute(img_ct)
myshow(sitk.LabelOverlay(img_255, seg), "Otsu Thresholding")

print(otsu_filter.GetThreshold() )
print(img_ct.GetSize())
print(seg.GetSize())

The [maximum entropy](https://www.sciencedirect.com/science/article/pii/0734189X85901252) method, on the other hand, choose a threhold value such that the entropies of distributions above and below the threshold value is maximised.

In [ ]:
MaxEntropy_Filter = sitk.MaximumEntropyThresholdImageFilter()
MaxEntropy_Filter.SetInsideValue(0)
MaxEntropy_Filter.SetOutsideValue(1)
seg = MaxEntropy_Filter.Execute(img_ct)
myshow(sitk.LabelOverlay(img_255, seg), "Maximum Entropy Thresholding")
print(MaxEntropy_Filter.GetThreshold())

One intersting applimcation is to define a region of interest (ROI) with minimal background with the help of the following filters:

* [sitk.LabelShapeStatisticsImageFilter](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1LabelShapeStatisticsImageFilter.html#details), and
* [sitk.RegionOfInterest](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1RegionOfInterestImageFilter.html)

In [ ]:
help(sitk.LabelShapeStatisticsImageFilter)

In [ ]:
label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
inside_value = 0
outside_value = 1
otsu_filter.SetInsideValue(inside_value)
otsu_filter.SetOutsideValue(outside_value)
label_shape_filter.Execute( otsu_filter.Execute(img_ct))
bounding_box = label_shape_filter.GetBoundingBox(outside_value)
print(bounding_box)

In [ ]:
help(sitk.RegionOfInterest)

In [ ]:
myORI = sitk.RegionOfInterest(img_ct, bounding_box[int(len(bounding_box)/2):], bounding_box[0:int(len(bounding_box)/2)])
myshow(myORI)

These filters compute a threshold based on some heuristic and apply that threshold to the input image using the [BinaryThresholdImageFilter](https://itk.org/Doxygen/html/classitk_1_1BinaryThresholdImageFilter.html).

Many other histogram based automatic thresholding filters are implemented in SimpleITK, including
* [Huang's thresholding method](https://www.sciencedirect.com/science/article/pii/0031320394E0043K): separates an image into foreground and background component by minimizing the measures of fuzziness,
* [Traingle](http://journals.sagepub.com/doi/pdf/10.1177/25.7.70454): constructs a line between the histogram peak and the farthest end of the histogram. The threshold is the point of maximum distance between the line and the hidtogramm.

## Region Growing Segmentation

It is reasonable that the same class of tissue would exhibit similar pixel intensity within the image. If a **seed** is given, we can group a class of pixels that are similar (by some statistical measures) and in the neighbourhood of the seed. In SimpleITK, thse are some of the filters that perform region growing segmentation:
* <b>ConnectedThreshold</b>: The neighboring voxel's intensity is within explicitly specified thresholds.
* <b>ConfidenceConnected</b>: The neighboring voxel's intensity is within the implicitly specified bounds $\mu\pm c\sigma$, where $\mu$ is the mean intensity of the seed points, $\sigma$ their standard deviation and $c$ a user specified constant.
* <b>VectorConfidenceConnected</b>: A generalization of the previous approach to vector valued images, for instance multi-spectral images or multi-parametric MRI. The neighboring voxel's intensity vector is within the implicitly specified bounds using the Mahalanobis distance $\sqrt{(\mathbf{x}-\mathbf{\mu})^T\Sigma^{-1}(\mathbf{x}-\mathbf{\mu})}<c$, where $\mathbf{\mu}$ is the mean of the vectors at the seed points, $\Sigma$ is the covariance matrix and $c$ is a user specified constant.


In [ ]:
seed = (100, 170)
seg = sitk.Image(img_ct.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(img_ct)
seg[seed] = 1
seg = sitk.BinaryDilate(seg, 3)
myshow(sitk.LabelOverlay(img_255, seg), "Initial Seed")
print(img_ct.GetPixel(seed))

Let us experiment what each filter does. Staring with the ConnectedThreshold filter:
<ul>
  <li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ConnectedThresholdImageFilter.html">ConnectedThreshold</a>: Label pixels that are connected to a seed and lie within a range of values.</li>
</ul>

In [ ]:
help(sitk.ConnectedThreshold)

In [ ]:
seg = sitk.ConnectedThreshold(img_ct, seedList=[seed], lower=230, upper=255)
myshow(sitk.LabelOverlay(img_255,seg), "Connected Threshold")

The ConnectedThreshold filter:
<ul>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ConfidenceConnectedImageFilter.html">ConfidenceConnected</a>: Segment pixels with similar statistics using connectivity.

This filter extracts a connected set of pixels whose pixel intensities are consistent with the pixel statistics of a seed point. The mean and variance across a neighborhood (8-connected, 26-connected, etc.) are calculated for a seed point. Then pixels connected to this seed point whose values are within the confidence interval for the seed point are grouped. The width of the confidence interval is controlled by the "Multiplier" variable (the confidence interval is the mean plus or minus the "Multiplier" times the standard deviation). If the intensity variations across a segment were gaussian, a "Multiplier" setting of 2.5 would define a confidence interval wide enough to capture 99% of samples in the segment.

After this initial segmentation is calculated, the mean and variance are re-calculated. All the pixels in the previous segmentation are used to calculate the mean the standard deviation (as opposed to using the pixels in the neighborhood of the seed point). The segmentation is then recalculated using these refined estimates for the mean and variance of the pixel values. This process is repeated for the specified number of iterations. Setting the "NumberOfIterations" to zero stops the algorithm after the initial segmentation from the seed point.

NOTE: the lower and upper threshold are restricted to lie within the valid numeric limits of the input data pixel type. Also, the limits may be adjusted to contain the seed point's intensity.</li>
</ul>

In [ ]:
help(sitk.ConfidenceConnected)

The VectorConfidenceConnected filter:
<ul>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1VectorConfidenceConnectedImageFilter.html">VectorConfidenceConnected</a>: same as the Confidence Connected Image filter but the the pixel type is of vector type.</li>
</ul>

As an example, if we have multi-model image of the same anatomy (T1 and T2 weighted of a brain, as an example), we can segment this image using statistics based on the multi-modality image.